In [2]:
import glob
import os
import sys
import numpy as np
import cv2
import random


In [ ]:
# # import yolo
# from IPython import display
# display.clear_output()

In [2]:
# import tensorflow
import time

In [2]:
import ultralytics
from ultralytics import YOLO
# yolo=YOLO("yolov8n.pt")


c:\Users\ASUS\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from PIL import Image

In [5]:
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla

In [ ]:
# def process(image):
#     i=np.array(image.raw_data)
#     i2=i.reshape((600,600,4))
#     i3=i2[:,:,:3]
#    # cv2.imshow("",i3)
#     # cv2.waitKey(1)
#     normi=(i3/255.0)
#     opi=cv2.cvtColor((normi*255).astype(np.uint8),cv2.COLOR_RGB2BGR)
#     return opi

In [6]:
actor_list=[]

In [22]:
#import random
# client=carla.Client('localhost',2000)
# client.set_timeout(2000.0)
# world=client.get_world()
# bpl=world.get_blueprint_library()
# bp=bpl.filter('model3')[0]
# print(bp)
# spp=random.choice(world.get_map().get_spawn_points())
# # sp=spp[0]
# vl=world.spawn_actor(bp,spp)
# actor_list.append(vl)
# vp=vl.get_transform()

client=carla.Client('localhost',2000)
client.set_timeout(2000.0)
world=client.get_world()
bpl=world.get_blueprint_library()
bp=bpl.filter('model3')[0]
print(bp)
spp=world.get_map().get_spawn_points()
sp=spp[10]
vl=world.spawn_actor(bp,sp)
actor_list.append(vl)

ActorBlueprint(id=vehicle.tesla.model3,tags=[vehicle, tesla, model3])


In [23]:
#control the vehicle:
def control_vehicle(traffic_light_state):
    control = carla.VehicleControl()
    if traffic_light_state == "red":
        control.throttle = 0.0
        control.brake = 1.0
    elif traffic_light_state == "green":
        control.throttle = 1.0
        control.brake = 0.0
    else:
        # Default behavior if the traffic light state is not known
        control.throttle = 0.5
        control.brake = 0.0
    vl.apply_control(control)

In [24]:
yolo = YOLO('yolov8n.pt')

# Secondary model (yolotraficlight) for traffic light detection
yolotraficlight = YOLO("C:/Users/ASUS/Downloads/CARLA_0.9.8/WindowsNoEditor/PythonAPI/examples/best1.pt")

import supervision as sv

rf = Roboflow(api_key="PqaW7bzyyhdeZntUMVOJ")
project = rf.workspace().project("object-recognition-dddrb")
model = project.version(2).model

def detect_traffic_light_state(image):
    # Perform traffic light detection using yolotraficlight
    result = yolotraficlight(image)[0]
    print("bla bla bla bla bla bla")
    # Determine traffic light state (green, red, yellow)
    for box in result.boxes:
        state_label = result.names[int(box.cls[0])] 
        print("Detected traffic light state:", state_label)
        return state_label
    return "Unknown"

# Callback function for processing each frame
def cm_callback(image):
    # Convert image to numpy array
    img = np.array(image.raw_data).reshape((600, 600, 4))
    img_color = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)  # Convert to BGR format
    
    # Perform object detection using YOLOv8 on color image
    result = yolo(img_color)[0]

    # Iterate over detected objects
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Check if the detected object is a traffic light
        label = result.names[int(box.cls[0])]
        if label == "traffic light":
            # Crop the region of interest containing the traffic light from the color image
            traffic_light_roi = img_color

            
            # Detect traffic light state using yolotraficlight
            traffic_light_state = detect_traffic_light_state(traffic_light_roi)
            traffic_light_state = detect_traffic_light_state(traffic_light_roi)
            control_vehicle(traffic_light_state)
            
            # Draw bounding box and label on the original image
            cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_color, label + ": " + traffic_light_state, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
        else:
            # Draw bounding box and label on the original image for non-traffic light objects
            cv2.rectangle(img_color, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_color, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
    
    # Save or display the annotated image
    frame_id = str(time.time())
    file_path = os.path.join('data_carla', f'dete_obj_{frame_id}.png')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    cv2.imwrite(file_path, img_color)
    #cv2.imshow("Annotated Image", img_color)
    #cv2.waitKey(1)  # Use 1 ms waitKey to update the display window

In [ ]:
#vl.set_autopilot(True)

In [ ]:
#vehicle controle whitout autopilote
#vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))

In [25]:
bp2=bpl.find('sensor.camera.rgb')
bp2.set_attribute("image_size_x",f'{600}')
bp2.set_attribute("image_size_y",f'{600}')
bp2.set_attribute('fov','110')
#position of the sensor
ssp=carla.Transform(carla.Location(x=2.5,z=0.7))
sn=world.spawn_actor(bp2,ssp,attach_to=vl)
actor_list.append(sn)

In [ ]:
#vehicle controle without autopilote
vl.apply_control(carla.VehicleControl(throttle=1.0,steer=0.0))
sn.listen(cm_callback)






Ultralytics YOLOv8.0.145  Python-3.7.0 torch-1.13.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)


Ultralytics YOLOv8.0.145  Python-3.7.0 torch-1.13.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients
Ultralytics YOLOv8.0.145  Python-3.7.0 torch-1.13.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients
0: 640x640 (no detections), 5810.3ms
Speed: 88.8ms preprocess, 5810.3ms inference, 70.3ms postprocess per image at shape (1, 3, 640, 640)

Ultralytics YOLOv8.0.145  Python-3.7.0 torch-1.13.1+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
0: 640x640 (no detections), 6318.1ms
Speed: 57.2ms preprocess, 6318.1ms inference, 52.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5646.0ms
Speed: 47.5ms preprocess, 5646.0ms inference, 63.3ms postprocess per image at shape (1, 3, 640, 640)

YOLOv8n summary (fuse

bla bla bla bla bla bla


Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients
0: 640x640 (no detections), 19317.3ms
Speed: 68.2ms preprocess, 19317.3ms inference, 50.1ms postprocess per image at shape (1, 3, 640, 640)


bla bla bla bla bla bla



0: 640x640 2 traffic_light_reds, 19308.7ms
Speed: 59.5ms preprocess, 9654.4ms inference, 21.0ms postprocess per image at shape (1, 3, 640, 640)



...

bla bla bla bla bla bla


0: 640x640 2 traffic_light_greens, 1 traffic_light_red, 18725.6ms
Speed: 112.6ms preprocess, 18725.6ms inference, 120.7ms postprocess per image at shape (1, 3, 640, 640)



bla bla bla bla bla bla
Detected traffic light state: traffic_light_green


0: 640x640 2 traffic_light_greens, 19276.9ms
Speed: 106.1ms preprocess, 19276.9ms inference, 95.9ms postprocess per image at shape (1, 3, 640, 640)



bla bla bla bla bla bla
Detected traffic light state: traffic_light_green


In [ ]:
#vl.destroy()
#sn.destroy()
for actor in actor_list:
     actor.destroy()

0: 640x640 (no detections), 13553.8ms
Speed: 105.1ms preprocess, 13553.8ms inference, 77.5ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 (no detections), 19861.8ms
Speed: 165.3ms preprocess, 19861.8ms inference, 81.8ms postprocess per image at shape (1, 3, 640, 640)



bla bla bla bla bla bla


0: 640x640 (no detections), 18932.6ms
Speed: 77.7ms preprocess, 18932.6ms inference, 11.7ms postprocess per image at shape (1, 3, 640, 640)



bla bla bla bla bla bla


...

0: 640x640 (no detections), 5836.9ms
Speed: 22.2ms preprocess, 2918.5ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)



bla bla bla bla bla bla


0: 640x640 (no detections), 4011.1ms
Speed: 30.5ms preprocess, 4011.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)


bla bla bla bla bla bla


: 